Validar y ajustar los valores gatillo previamente calculados para cada provincia, asegurándose de que:

Coherencia Estadística: Los valores gatillo no sean valores atípicos extremos que puedan distorsionar análisis posteriores. Rango Razonable: Al limitar los valores gatillo dentro de tres desviaciones estándar de la media, se reduce la influencia de posibles datos anómalos o errores en los datos.

In [ ]:
import pandas as pd
import numpy as np

# Cargar los datos originales y los valores gatillo desde archivos CSV.
# Asegúrate de que los archivos 'dataset_modificado.csv' y 'gatillos_provincia_promedio_ultimos_2_anos.csv'
# estén en la misma carpeta que este script o cuaderno.
ruta_original = "dataset_modificado.csv"
ruta_gatillos = "gatillos_provincia_promedio_ultimos_2_anos.csv"

# Leer los datos en DataFrames de pandas.
df_original = pd.read_csv(ruta_original)
df_gatillos = pd.read_csv(ruta_gatillos)

# Crear un diccionario para almacenar los nuevos valores gatillo ajustados.
nuevos_gatillos = {
    "provincia": [],
    "gatillo_temp_max_max": [],
    "gatillo_humedad_max": []
}

# Iterar sobre cada provincia para ajustar sus valores gatillo.
for provincia in df_gatillos['provincia'].unique():
    # Filtrar los datos originales para la provincia actual.
    df_provincia_original = df_original[df_original['provincia'] == provincia]
    
    # Calcular estadísticas descriptivas (media y desviación estándar) para variables climáticas relevantes.
    temp_max_max_mean = df_provincia_original['temp_max_max'].mean()
    temp_max_max_std = df_provincia_original['temp_max_max'].std()
    
    humedad_max_mean = df_provincia_original['humedad_max'].mean()
    humedad_max_std = df_provincia_original['humedad_max'].std()
    
    # Obtener los valores gatillo actuales de la provincia.
    gatillo_temp_max_max = df_gatillos.loc[df_gatillos['provincia'] == provincia, 'gatillo_temp_max_max'].values[0]
    gatillo_humedad_max = df_gatillos.loc[df_gatillos['provincia'] == provincia, 'gatillo_humedad_max'].values[0]
    
    # Definir el rango razonable (media ± 3 veces la desviación estándar).
    rango_temp_max_max_inf = temp_max_max_mean - 3 * temp_max_max_std
    rango_temp_max_max_sup = temp_max_max_mean + 3 * temp_max_max_std
    
    rango_humedad_max_inf = humedad_max_mean - 3 * humedad_max_std
    rango_humedad_max_sup = humedad_max_mean + 3 * humedad_max_std
    
    # Ajustar el valor gatillo de temperatura máxima si está fuera del rango razonable.
    if not (rango_temp_max_max_inf <= gatillo_temp_max_max <= rango_temp_max_max_sup):
        gatillo_temp_max_max = np.clip(
            gatillo_temp_max_max, rango_temp_max_max_inf, rango_temp_max_max_sup
        )
    
    # Ajustar el valor gatillo de humedad máxima si está fuera del rango razonable.
    if not (rango_humedad_max_inf <= gatillo_humedad_max <= rango_humedad_max_sup):
        gatillo_humedad_max = np.clip(
            gatillo_humedad_max, rango_humedad_max_inf, rango_humedad_max_sup
        )
    
    # Almacenar los valores ajustados en el diccionario.
    nuevos_gatillos['provincia'].append(provincia)
    nuevos_gatillos['gatillo_temp_max_max'].append(gatillo_temp_max_max)
    nuevos_gatillos['gatillo_humedad_max'].append(gatillo_humedad_max)

# Convertir el diccionario en un DataFrame para facilitar su manejo.
df_nuevos_gatillos = pd.DataFrame(nuevos_gatillos)

# Guardar los nuevos valores gatillo ajustados en un archivo CSV.
ruta_nuevos_gatillos = "nuevos_valores_gatillo.csv"
df_nuevos_gatillos.to_csv(ruta_nuevos_gatillos, index=False)

# Mostrar los primeros registros de los nuevos valores gatillo para verificar los resultados.
print("Nuevos valores gatillo ajustados:")
print(df_nuevos_gatillos.head())
